<a href="https://colab.research.google.com/github/hannape/Gruba-kreska/blob/main/1_Poprawienie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jeszcze przed wczytywaniem danych, trzeba sprawdzić co się dzieje w etykietach, które są negatywne, które pozytywne, czy nie ma literówek, spacji itd. By na pewno było wiadomo która etykieta idzie jako negative a która jako positive


In [1]:
# Wczytywanie google drive'a
from google.colab import drive
drive.mount('/content/drive')
from __future__ import print_function

Mounted at /content/drive


In [24]:
import os
import contextlib
import numpy as np
import wave
import math

In [25]:
# Parametry które będą wchodzić do funkcji

path_test1618_txt = 'drive/My Drive/poprawione etykiety 24112020/testowe/'
path_train161718_txt =  'drive/My Drive/poprawione etykiety 24112020/treningowe/'
path_test1618_wav = 'drive/My Drive/testowe_1618/'
path_train161718_wav =  'drive/My Drive/treningowe94_161718/'
balance_type = ['balanced', 'valid', 'full_rec']  # na potrzeby treningu, valida, testu
chunk_length_ms = 500 # 500 ms
chunk_overlap = 150 # 150 ms

Krok 1. Sprawdzić jakie etykiety w ogóle mam we wszystkich nagraniach. By wiedzieć co wyciąć i uznać za negatywne, a co uznać za pozytywne (bo moga być jakieś literówki, podwójne spacje, itd)



In [66]:
def my_read_labels(label_name,path_txt):
    '''Wczytuje etykiety czasowe z pliku labels.txt w folderze train.
    
    Returns:
        ndarray: Tablica z etykietami czasowymi zawierająca kolumny:  sekunda początku dźwięku, sekunda końca dźwięku, gatunek.
    '''
    labels = []
    with open(os.path.join(path_txt, label_name + '.txt'), 'r') as file:
        text = file.read()
        for line in text.split('\n')[0:]:
            if len(line) > 1:
                start, stop, spec = line.split('\t')
                #print(start)
                start, stop, spec = float(start), float(stop), str(spec),
                labels.append([start, stop, spec])
    return np.array(labels)


def my_load_train(path_wav, path_txt): 
  X_train, y_train = [], []
  recording_labels_all = []
  in_which_recording = []

  rec_files = [file_name for file_name in os.listdir(path_wav) if file_name.endswith('.wav')]
  years = ['2016','2017','2018']
  files_sorted = []

  for year in years:
    i_201x = [i for i,s in enumerate(rec_files) if year in s]
    n_201x = sorted(rec_files[index] for index in i_201x)
    files_sorted = files_sorted + n_201x

  print(files_sorted)
  for file_name in files_sorted:
      
      print("------------Analiza nagrania: " + file_name + "-----------")
      recording_id = str(file_name.split('.')[0])
      '''if (file_name=="BUK4_20171001_020404a.wav"):
        rok = "7"
      if (file_name=="BUK4_20181029_235604.wav"):
        rok = "8"  
      source_npz = "/content/drive/My Drive/nobirds_chunks/repr7_201" + rok + "/"
      ## wczytanie plików, skąd można wziąc nr próbek losowych do balanced dataset
      dane1 = np.load(source_npz + 'Kopia '+ recording_id + '_7rep.npz' )
      nobirds_chunks = dane1["nobirds_chunks"] 
      print(np.shape(nobirds_chunks)) '''
      
      fname = path_wav + file_name      #print(fname)
      with contextlib.closing(wave.open(fname,'r')) as f:
          frames = f.getnframes()
          rate = f.getframerate()
          duration = frames / float(rate)
          recording_duration = math.ceil(duration)
          print("Czas trwania nagrania - w sekundach: " + str(recording_duration))
    
      recording_id = (file_name.split('.')[0])       #print(recording_id)
      recording_labels = my_read_labels(recording_id, path_txt)    
      #print(recording_labels[:,2])
      #print(type(recording_labels[:,2]))
      #print(np.shape(recording_labels))
      #print((recording_labels[:,2].split))
      #a = recording_labels[:,2]
      if np.shape(recording_labels)[0]>0:
        recording_labels_all.extend(recording_labels[:,2])
        in_which_recording.extend([file_name]*np.shape(recording_labels[:,2])[0])
      print(np.shape(recording_labels_all))
      print(np.shape(in_which_recording))
  return (recording_labels_all, in_which_recording)

In [ ]:
#@title
## Na razie nieużywana funkcja, będzie użyta potem

def my_map_seconds_to_y(labels, recording_duration):  ## Chcemy 500 ms z 150 ms overlapem
    '''Tworzy etykiety dla każdego kolejnego chunksa. 1 gdy co najmniej 4 ms etykiety się znajdują w chunksie (gdy urwane dźwięki na brzegach, <4ms - to 0).
    
    Args:
        labels (ndarray): Tablica z etykietami, której 1 kolumna oznacza początek, a druga - koniec nagrania.
    Returns:
        ndarray: Tablica z binarnymi etykietami.
    '''
    #calls_to_cut = ['t', 't?', 't ','t  ', 'g', 'czapla', 'gh', 'g cz', 'puszczyk','gaski','g?','mewa?','zwierzak?','high freq','g niskie','??? mysz']
    
    '''
    duration_in_ms = recording_duration*1000
    nr_of_chunks =  1 + (duration_in_ms - chunk_length_ms) / (chunk_length_ms - chunk_overlap)
    print("chunksy!!")
    print(nr_of_chunks)  
    
    y = [0] * math.ceil(nr_of_chunks)             # recording_duration 
    y_restrictive = [0] * math.ceil(nr_of_chunks) # recording_duration
    for s in range(math.ceil(nr_of_chunks)):
        for l in labels:
            if my_check_voices(s*0.35, l):
                y[s] = 1
                #print(s*0.35)
                if l[2] in calls_to_cut:
                  y[s] = 0
                 # print("Labele1: " + l[2]) 
            if my_check_voices(s*0.35, l, 0.004): # 0.02
                y_restrictive[s] = 1
                if l[2] in calls_to_cut:
                  y_restrictive[s] = 0
                  #print("wyciety glos:")
                  #print(l[2])
                  
                 # print("Labele2: " + l[2]) 
        if y[s] != y_restrictive[s]:
            y[s] = 0 # -1 jeśli mniej niż 4 ms fragment, to zakładamy że tam go nie ma...          
   
    return y    
    '''

In [33]:
calls_0 = ['t', 'g', 'czapla', 'gh', 'puszczyk']                                    # etykiety które są negatywne
calls_1 = ['d', 'd?', 'k', 'k?', 'kwiczol', 'r','r?', 's', 's?', 'skowronek', 'ni'] # calls of interest
calls_unknown = ['???','??? mysz', '??? high freq']                                 # unknown, nie wiem czy to głos czy nie, więc będę wyrzucać te chunksy to zawierające

**TRENINGOWE**

In [ ]:
recording_labels_all, in_which_recording = my_load_train(path_train161718_wav, path_train161718_txt)  # wyznaczenie wszystkich labeli z próbek treningowych

In [29]:
from collections import Counter
counted_labels = Counter(recording_labels_all)

**Fragment tekstowy opisujący jak wyglądały etykiety w trainie wcześniej i teraz**

Bez poprawy, stare etykiety wyglądały tak:

```
Counter({'': 3,
         '?': 1,
         '???': 61,
         '??? mysz': 8,
         '?????': 1,    - głosy ok, 1
         '??????': 5,   - głosy ok, 1
         'a?': 11,
         'czapla': 1,
         'd': 145,
         'd ': 1,
         'd?': 17,
         'd???': 1,
         'g': 55,
         'g cz': 2,
         'g niskie': 1,
         'g?': 2,
         'g???': 1,
         'gaski': 2,
         'gh': 73,
         'high freq': 1,
         'k': 612,
         'k?': 42,
         'k???': 1,
         'kwiczol': 14,
         'mewa?': 3,
         'ni': 74,
         'puszczyk': 7,
         'r': 27,
         'r?': 49,
         'reg?': 2,
         's': 3413,
         's ': 1,
         's?': 120,
         's???': 4,
         's?r?': 2,
         'skowronek': 15,
         't': 53,
         't?': 9,
         'zwierzak?': 1})

```

Trzeba sprawdzić i ustalić co zrobić z  '?' , '???', '?????', '??????', 'a?', 'g?', 'g???'

Pierwsza poprawa, 24.11:

```
Counter({'': 4,
         '???': 227,
         '??? mysz': 7,
         'czapla': 1,
         'd': 146,
         'd ': 1,
         'd?': 21,
         'g': 258,
         'g cz': 2,
         'gaski': 1,
         'gh': 99,
         'high freq': 1,
         'k': 626,
         'k  ': 1,
         'k?': 46,
         'kwiczol': 14,
         'ni': 186,
         'puszczyk': 1,
         'r': 37,
         'r?': 52,
         's': 3523,
         's ': 1,
         's?': 127,
         'skowronek': 26,
         't': 1344,
         't?': 3})
         
```

poprawa poprawionych 24.11, wersja ostateczna labeli:
```
Counter({'???': 227,
         '??? high freq': 1,
         '??? mysz': 7,
         'czapla': 3,
         'd': 147,
         'd?': 21,
         'g': 260,
         'gh': 99,
         'k': 628,
         'k?': 46,
         'kwiczol': 14,
         'ni': 187,
         'puszczyk': 1,
         'r': 37,
         'r?': 52,
         's': 3524,
         's?': 128,
         'skowronek': 26,
         't': 1346})
         ```


In [ ]:
# fragment do znalezienia w jakich nagraniach i próbkach jest dana etykieta

elem = 'k'

index_pos_list = [ i for i in range(len(recording_labels_all)) if recording_labels_all[i] == elem ]
res_list = [in_which_recording[i] for i in index_pos_list] 
print('Indexes of all occurrences of label ', elem,' in the list are : ', index_pos_list)
print('They are in recordings: ', res_list)
Counter(res_list)

In [ ]:
#@title
file_name = 'BUK4_20171001_020404a'
abc = ['aaa', 'wefd']
abc.extend([file_name]*4)
print(abc)

In [61]:
count_positives = 0
count_negatives = 0
count_unknown = 0

for calls in calls_1:
  count_positives += counted_labels[calls]

for calls in calls_0:
  count_negatives += counted_labels[calls]

for calls in calls_unknown:
  count_unknown += counted_labels[calls]    

cztery_gat = counted_labels['s']+counted_labels['s?'] + counted_labels['k']+counted_labels['k?'] +counted_labels['d']+counted_labels['d?'] + counted_labels['r']+counted_labels['r?']

print('Liczba pozytywnych etykiet: ', count_positives)
print('Liczba negatywnych etykiet: ', count_negatives)
print('Liczba nieznanych etykiet: ', count_unknown)
print('Stosunek gatunków w zbiorze treningowym:')
print('-   bezwzględne s:k:d:r - ', counted_labels['s']+counted_labels['s?'],':', counted_labels['k']+counted_labels['k?'],':', counted_labels['d']+counted_labels['d?'],':', counted_labels['r']+counted_labels['r?'])
print('-   procentowe s:k:d:r - ', round(100*(counted_labels['s']+counted_labels['s?'])/count_positives,1),':',
      round(100*(counted_labels['k']+counted_labels['k?'])/count_positives,1),':',
      round(100*(counted_labels['d']+counted_labels['d?'])/count_positives,1),':',
      round(100*(counted_labels['r']+counted_labels['r?'])/count_positives,1))
print('te 4 gatuny odpowiadają za ', round(100*cztery_gat/count_positives,2),'% wszystkich pozytywnych etykiet w zbiorze')

Liczba pozytywnych etykiet:  4810
Liczba negatywnych etykiet:  1709
Liczba nieznanych etykiet:  235
Stosunek gatunków w zbiorze treningowym:
-   bezwzględne s:k:d:r -  3652 : 674 : 168 : 89
-   procentowe s:k:d:r -  75.9 : 14.0 : 3.5 : 1.9
te 4 gatuny odpowiadają za  95.28 % wszystkich pozytywnych etykiet w zbiorze


**TESTOWE**

In [ ]:
recording_labels_all, in_which_recording = my_load_train(path_test1618_wav, path_test1618_txt)

In [74]:
from collections import Counter
counted_labels = Counter(recording_labels_all)

In [75]:
print(counted_labels)

Counter({'k': 373, 't': 360, 's': 264, '???': 89, 'g': 72, 'ni': 57, 's?': 33, 'k?': 26, 'r': 16, 'd': 14, 'd?': 6, 'r?': 4, 'skowronek?': 1})


In [76]:
count_positives = 0
count_negatives = 0
count_unknown = 0

for calls in calls_1:
  count_positives += counted_labels[calls]

for calls in calls_0:
  count_negatives += counted_labels[calls]

for calls in calls_unknown:
  count_unknown += counted_labels[calls]    

cztery_gat = counted_labels['s']+counted_labels['s?'] + counted_labels['k']+counted_labels['k?'] +counted_labels['d']+counted_labels['d?'] + counted_labels['r']+counted_labels['r?']

print('Liczba pozytywnych etykiet: ', count_positives)
print('Liczba negatywnych etykiet: ', count_negatives)
print('Liczba nieznanych etykiet: ', count_unknown)
print('Stosunek gatunków w zbiorze testowym:')
print('-   bezwzględne s:k:d:r - ', counted_labels['s']+counted_labels['s?'],':', counted_labels['k']+counted_labels['k?'],':', counted_labels['d']+counted_labels['d?'],':', counted_labels['r']+counted_labels['r?'])
print('-   procentowe s:k:d:r - ', round(100*(counted_labels['s']+counted_labels['s?'])/count_positives,1),':',
      round(100*(counted_labels['k']+counted_labels['k?'])/count_positives,1),':',
      round(100*(counted_labels['d']+counted_labels['d?'])/count_positives,1),':',
      round(100*(counted_labels['r']+counted_labels['r?'])/count_positives,1))
print('te 4 gatuny odpowiadają za ', round(100*cztery_gat/count_positives,2),'% wszystkich pozytywnych etykiet w zbiorze')

Liczba pozytywnych etykiet:  793
Liczba negatywnych etykiet:  432
Liczba nieznanych etykiet:  89
Stosunek gatunków w zbiorze testowym:
-   bezwzględne s:k:d:r -  297 : 399 : 20 : 20
-   procentowe s:k:d:r -  37.5 : 50.3 : 2.5 : 2.5
te 4 gatuny odpowiadają za  92.81 % wszystkich pozytywnych etykiet w zbiorze


In [79]:
count_positives2 = count_positives - 314
cztery_gat2 = cztery_gat - 314
print('Jeśli byśmy usunęli ze zbioru testowego nagranie BUK5_20161101_002104a i BUK5_20161101_002104b, to byśmy otrzymali:')
print('-   bezwzględne s:k:d:r - ', counted_labels['s']+counted_labels['s?'],':', counted_labels['k']+counted_labels['k?']-314,':', counted_labels['d']+counted_labels['d?'],':', counted_labels['r']+counted_labels['r?'])
print('-   procentowe s:k:d:r - ', round(100*(counted_labels['s']+counted_labels['s?'])/count_positives2,1),':',
      round(100*(counted_labels['k']+counted_labels['k?']-314)/count_positives2,1),':',
      round(100*(counted_labels['d']+counted_labels['d?'])/count_positives2,1),':',
      round(100*(counted_labels['r']+counted_labels['r?'])/count_positives2,1))
print('te 4 gatuny odpowiadają za ', round(100*cztery_gat2/count_positives2,2),'% wszystkich pozytywnych etykiet w zbiorze')

Jeśli byśmy usunęli ze zbioru testowego nagranie BUK5_20161101_002104a i BUK5_20161101_002104b, to :
-   bezwzględne s:k:d:r -  297 : 85 : 20 : 20
-   procentowe s:k:d:r -  62.0 : 17.7 : 4.2 : 4.2
te 4 gatuny odpowiadają za  88.1 % wszystkich pozytywnych etykiet w zbiorze


In [82]:
duration_in_ms = 1799*1000
nr_of_chunks =  1 + (duration_in_ms - chunk_length_ms) / (chunk_length_ms - chunk_overlap)
print(nr_of_chunks)

5139.571428571428
